# Interval based time series classification in sktime

Interval based approaches look at phase dependant intervals of the full series,
calculating summary statistics from selected subseries to be used in classification.
They follow the ensemble model of random forest: each base classifier is a form of
decision tree. Diversification is achieved by randomly sampling intervals

<img src="./img/ensemble.png" width="700" alt="ensemble.">

Currently, there are five univariate interval based approaches are implemented in
sktime. Time Series Forest (TSF) \[1\], the Random Interval Spectral Ensemble (RISE)
\[2\], Supervised Time Series Forest (STSF) \[3\], the Canonical Interval Forest
(CIF) \[4\]
 and the Diverse Representation Canonical Interval Forest (DrCIF) [5]. Both CIF and
 DrCIF have multivariate capabilities.

In this notebook, we will demonstrate how to use these classifiers on the ItalyPowerDemand and BasicMotions datasets.


## Imports and Load data

In [2]:
from sklearn import metrics

from sktime.classification.interval_based import (
    CanonicalIntervalForest,
    DrCIF,
    RandomIntervalSpectralEnsemble,
    SupervisedTimeSeriesForest,
    TimeSeriesForestClassifier,
)
from sktime.datasets import load_basic_motions, load_italy_power_demand

X_test, y_test = load_italy_power_demand(split="test")
X_train, y_train = load_italy_power_demand(split="train")
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train")
X_test_mv, y_test_mv = load_basic_motions(split="test")

X_train_mv = X_train_mv[:50]
y_train_mv = y_train_mv[:50]
X_test_mv = X_test_mv[:50]
y_test_mv = y_test_mv[:50]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)

SyntaxError: invalid syntax (634118166.py, line 14)

## 1. Time Series Forest (TSF)

TSF[1] is the simplest interval based tree based ensemble. For each tree,sqrt(series_length)
intervals are selected with a random position and length. The same interval offsets are applied to all series. For each interval, three summary statistics (the mean, variance and slope) are extracted  and concatenated into a feature vector. This feature vector is used to build the tree, and features extracted from the same intervals are used to make predictions. The ensembles predictions are made using a majority vote of all trees. The TSF base classifier is a modified decision tree classifier referred to as a time series tree, which considers all attributes at each node and uses a metric called margin gain to break ties.

In [ ]:
tsf = TimeSeriesForestClassifier(n_estimators=50, random_state=47)
tsf.fit(X_train, y_train)
tsf_preds = tsf.predict(X_test)
print(f"TSF Accuracy: {metrics.accuracy_score(y_test, tsf_preds)}")

## 2. Random Interval Spectral Ensemble (RISE)

First developed for the HIVE-COTE ensemble, RISE [2] is an
interval based tree ensemble that uses spectral features. Unlike TSF, RISE selects a single random interval for each base classifier. The periodogram and auto-regression function are calculated over each randomly selected interval, and these features are concatenated into a feature vector, from which a tree is built. RISE was primarily designed for use with audio problems, where spectral features are more likely to be discriminatory.

In [ ]:
rise = RandomIntervalSpectralEnsemble(n_estimators=50, random_state=47)
rise.fit(X_train, y_train)

rise_preds = rise.predict(X_test)
print(f"RISE Accuracy: {metrics.accuracy_score(y_test, rise_preds)}")

## 3. Supervised Time Series Forest (STSF)

STSF [3] is an interval based tree ensemble that includes a supervised method for
extracting intervals. Intervals are found and extracted for a periodogram and first order differences representation as well as the base series. STSF introduces bagging for each tree and extracts seven simple summary statistics from each interval. For each tree, an initial split point for the series is randomly selected. For both of these splits, the remaining subseries is cut in half, and the half with the higher Fisher score is retained as an interval. This process is then run recursively using higher scored intervals until the series is smaller than a threshold. This is repeated for each of the seven summary statistic features, with the extracted statistic being used to calculate the Fisher score.

In [ ]:
stsf = SupervisedTimeSeriesForest(n_estimators=50, random_state=47)
stsf.fit(X_train, y_train)

stsf_preds = stsf.predict(X_test)
print("STSF Accuracy: " + str(metrics.accuracy_score(y_test, stsf_preds)))

## 6. Canonical Interval Forest (CIF)

CIF [4] is another extension of TSF, that improves
accuracy by integrating more informative features and by increasing diversity. Like
other interval approaches, CIF is an ensemble of decision tree classifiers built on
features extracted from phase dependent intervals. Alongside the mean, standard
deviation and slope, CIF also extracts the catch22 features [5].  Intervals remain
randomly generrated, with each tree selecting  $\sqrt{m}\sqrt{d}$ intervals. To add additional diversity to the ensemble, $a$ attributes out of the pool of $25$ are randomly selected for each tree.
    The extracted features are concatenated into a $k \cdot a$ length vector for each data series and used to build the tree. For multivariate data, CIF randomly selects the dimension used for each interval.

### Univariate

In [ ]:
cif = CanonicalIntervalForest(n_estimators=50, att_subsample_size=8, random_state=47)
cif.fit(X_train, y_train)

cif_preds = cif.predict(X_test)
print("CIF Accuracy on Italy: " + str(metrics.accuracy_score(y_test, cif_preds)))
cif_m = CanonicalIntervalForest(n_estimators=50, att_subsample_size=8, random_state=47)
cif_m.fit(X_train_mv, y_train_mv)

cif_m_preds = cif_m.predict(X_test_mv)
print("CIF Accuracy on Motion: " + str(metrics.accuracy_score(y_test_mv, cif_m_preds)))

## 6. Diverse Representation Canonical Interval Forest (DrCIF)

(DrCIF) [6] incorporates two new series representations: the
periodograms (also used by RISE and STSF) and first order differences (also used by STSF). For each of the three representations, $(4 + \sqrt{r}\sqrt{d})/3$ phase dependent intervals are randomly selected and concatenated into a feature vector, where $r$ is the length of the series for a representation.

In [ ]:
drcif = DrCIF(n_estimators=5, att_subsample_size=10, random_state=47)
drcif.fit(X_train, y_train)

drcif_preds = drcif.predict(X_test)
print("DrCIF Accuracy: " + str(metrics.accuracy_score(y_test, drcif_preds)))
drcif.fit(X_train_mv, y_train_mv)
drcif_preds = drcif.predict(X_test_mv)
print("DrCIF Accuracy: " + str(metrics.accuracy_score(y_test_mv, drcif_preds)))


#### References:

\[1\] Deng, H., Runger, G., Tuv, E., & Vladimir, M. (2013). A time series forest for classification and feature extraction. Information Sciences, 239, 142-153.

\[2\] Flynn, M., Large, J., & Bagnall, T. (2019). The contract random interval spectral ensemble (c-RISE): the effect of contracting a classifier on accuracy. In International Conference on Hybrid Artificial Intelligence Systems (pp. 381-392). Springer, Cham.

\[3\] Cabello, N., Naghizade, E., Qi, J., & Kulik, L. (2020). Fast and Accurate Time Series Classification Through Supervised Interval Search. In IEEE International Conference on Data Mining.

\[4\] Middlehurst, M., Large, J., & Bagnall, A. (2020). The Canonical Interval Forest (CIF) Classifier for Time Series Classification. arXiv preprint arXiv:2008.09172.

\[5\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[6\] Middlehurst M, Large J, Flynn M, Lines J, Bostrom A, Bagnall A (2021) HIVE-COTE 2.0: a new meta ensemble for time series classification. Machine Learning 110:3211–3243